1️⃣ Instalar dependencias

In [ ]:
!pip install "unsloth[cuda12]" datasets accelerate bitsandbytes trl

2️⃣ Cargar el modelo

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-bnb-4bit",  # modelo base
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=None,
)

3️⃣ Preparar los datos de entrenamiento

Ejemplo mínimo con un dataset estilo instrucción-respuesta.

In [ ]:
from datasets import Dataset

data = [
    {"instruction": "Explica qué es el reciclaje.", "output": "El reciclaje consiste en reutilizar materiales para reducir desechos."},
    {"instruction": "¿Qué es Python?", "output": "Python es un lenguaje de programación interpretado y multiparadigma."},
]

dataset = Dataset.from_list(data)

def format_prompt(example):
    return f"Instrucción: {example['instruction']}\nRespuesta: {example['output']}"

dataset = dataset.map(lambda e: {"text": format_prompt(e)})

4️⃣ Activar modo entrenamiento

In [ ]:
model = FastLanguageModel.for_training(model)

5️⃣ Configurar entrenamiento con LoRA (ajuste liviano)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    packing=False,
    args=TrainingArguments(
        output_dir="mistral-finetuned",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=2,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_strategy="epoch",
        optim="paged_adamw_8bit",
    ),
)
trainer.train()

6️⃣ Guardar el modelo ajustado


In [ ]:
model.save_pretrained("mistral_finetuned_unsloth")
tokenizer.save_pretrained("mistral_finetuned_unsloth")

7️⃣ Probar el modelo

In [ ]:
inputs = tokenizer("Instrucción: ¿Qué es el reciclaje?\nRespuesta:", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Ventajas de usar unsloth:

Usa quantization 4-bit → ahorra RAM VRAM.

Entrena mucho más rápido (hasta 2–3× que HF normal).

Soporta LoRA y QLoRA sin configuración complicada.

Compatible con Hugging Face y TRL.

# 🧩 ¿Qué es LoRA?

LoRA (Low-Rank Adaptation) es una técnica para ajustar modelos grandes sin modificar todos sus parámetros.

👉 En lugar de reentrenar los miles de millones de parámetros del modelo, LoRA:

Congela el modelo original.
Los pesos base del modelo (por ejemplo, de Mistral o Llama) no se tocan.

Agrega pequeñas capas entrenables (matrices de baja dimensión) dentro de las capas del modelo.
Esas matrices (llamadas A y B) aprenden los ajustes necesarios para la nueva tarea.

Durante la inferencia, el modelo base + las capas LoRA producen el resultado ajustado.

🔹 Ventaja:
Solo entrenas un 1–2% del total de parámetros.
➡️ Mucho más rápido, menos memoria y puedes cambiar tareas fácilmente solo cargando distintas capas LoRA.

📦 Ejemplo:

“Tomo Mistral 7B, lo adapto con LoRA para responder preguntas legales en Chile.”
Si luego quiero un modelo para educación, solo cambio las capas LoRA por otras.

# ⚙️ ¿Qué es QLoRA?

QLoRA (Quantized LoRA) es una versión aún más optimizada de LoRA.

Parte de la misma idea, pero cuantiza el modelo base antes del entrenamiento.

“Cuantizar” significa representar los pesos con menos bits (por ejemplo, 4-bit en lugar de 16 o 32).

Así el modelo ocupa mucho menos VRAM (puedes entrenar un modelo de 7B en una GPU de 12GB o incluso menos).

Entonces:

🔸 LoRA = capas adicionales pequeñas.

🔸 QLoRA = LoRA + modelo base en 4-bit (más liviano).